In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/home/ddalton/miniconda3/envs/flan-t5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.10it/s]


In [2]:
import re

prompt_1 = """Review Text: "%s"

Task: Your task is to convert this text into Basic Note Type (front/back) Anki flashcards. Prioritize information regarding the imaging features of diseases, unique imaging findings, and methods of differentiating similar disease entities. Ensure that each flashcard is clearly written, and adheres to the specified formatting and reference criteria.

Formatting Criteria:

- Construct a table with three columns: "Front", “Back”, "Number".
- Each row of the "Front" column should contain a single question testing the imaging features of disease, unique imaging findings, and methods of differentiating similar disease entities.
- The “Back” column should contain the succinct answer to the question in the corresponding row of the “Front” column.
- The "Number" column will serve to number each row, facilitating feedback.

Reference Criteria for each "Statement":

- Each flashcard should test a single concept.
- Each flashcard MUST be able to stand alone. Include the subject of the flashcard somewhere in the text.
- Keep ONLY simple, direct questions in the "Front" column.
- Clear concise language but if required give plenty of context. 
- Output csv format like the example below. 
- Output at least %d rows of question/answers.

Example:

Front;Back;Number
"How is necrotic tissue identified in acute pancreatitis on a CT scan?";"Lack of contrast enhancement.";1
"Why should people create their own examples?";"Because Jake is too tired to think of good examples.";2

"""

text = "Let's set our eyes in a very important function,\nthe exponential function. For this, we need to\ndefine Euler's number. Euler's number is quite a\nspecial number of mathematics. It appears in many branches and it can be defined\nin many ways. One of them is by the numerical\nvalue 2.71828182, etc. The decimal never terminates because a number is irrational, which means it\ncannot be expressed as a ratio between two integers. Another way to define it is using the expression\n1 plus 1 over n to the n. Let's look at this expression for several\nvalues of n. For 1, it's 1 plus 1 over 1 to\nthe one which is two. For 10 is 1 plus a tenth\nto the 10, which is 2.594. For 100 is 2.705. For 1,000 is 2.717. As you notice, the\nmore we increase n, the more these converges\nto a certain number. The actual number that this\ngets closer and closer is e, 2.71828182, etc. That's how we're going to define the number e and e has a\nvery particular property. If you consider the function\nf of x equals e to the x, that function is\nits own derivative. The derivative of f\nof x is also f of x. That particular property\nmakes sure that e appears in many places in science,\nstatistics, probability, etc. Now, the way I like to define\ne is using interest rates. Let's look at the\nfollowing problem. Let's say that you're\ntrying to find the best bank where\nto put your money. Bank 1 says the following. Bank 1 says, I'm not going to give you any money for a year. But after a year, I will give you as interest,\n100 percent of your money. That means all your\nmoney once a year. Means if you put in\n$5 today in a year, they give you another $5. That's pretty good, but\nyou continue looking. Let's go to bank number 2. Bank number 2 says, well, I'm going to give you 50 percent of your money every six months. If you put in $10\ntoday, in six months, you're going to have $15 because bank 2 gives you half of\nyour money twice a year. Bank 3 is slightly different. This one says, I'm\ngoing to give you 1/3 of your money\nthree times a year, so 33 percent every four months. In summary, bank 1 gives you\nall your money once a year, bank 2 gives you half\nfor money twice a year, and bank 3 gives you a third of your money three times a year. The question is,\nwhich bank is better? Let's take five seconds\nfor you to figure out. If you need more\ntime, feel free to pause the video and\nthink about it. This is a very\nimportant problem. The answer is that bank 3 is\nthe best bank of the three. Why? Well, let's\nsay you have $1. So bank 1 after a year\ngives you another one, so you end up with $2. Now, the question is, what are you going to\nend up with in bank 2 and what are you going\nto end up with in bank 3? In other words, what's the\npercentage that you end up getting in a year in\nbank 2 and bank 3. Let's take a look. Bank 1, you have $1 now. In a year, you're going\nto have your dollar plus the extra dollar that bank 1 gives you of\ninterests, that's $2. We can express the two\nas 1 plus 1 to the 1. That's going to be\nimportant later. In bank 2, now, you have\n$1 and in six months, you have $1 plus a half. In one year, you're going to have $8\nplus a half of that. In six months, you end up with\n1 plus 1/2, which is 1.5. In one year, you\nend up with 2.25, which is 1 plus 1/2 squared is because you're adding\nhalf of your money, which means you're\nmultiplying it by 1 plus 1 half every time and 2.25 is bigger than two because\nyour dollar got doubled. But this amount\nof money that you earned actually earned\nmore money later. This is called accrued\ninterests into money that gains interests and then that\ninterests gains interests. That's why you did better\nwith bank 2 than with bank 1. Now, let's take a\nlook at bank 3. In bank 3, you start\nwith $1 right now. In four months, you have $1\nplus 1/3, which is 1.33. In eight months,\nyou have your 1 and 1/3 plus a 1/3 of that. That's 1.77. In one year you have 1.77 plus 1/3 of\nthat, which is 2.37. Again, you did well\nbecause this interest over here accrued interests\nand accrued more interest. This interest over here\naccrued more interests. Your money is making more money. At the beginning of\nthe year, you had $1, then you have 1 plus 1/3, then you had 1 plus 1/3 squared, and then you had\n1 plus 1/3 cube, and that becomes 237. In short, in bank 1 after\none year, you have $2. In bank 2 after one\nyear, you have 2.25. In bank 3 after one\nyear, you have 2.37. That's why the money\nis better in bank 3. This money actually\nmultiplies a lot. Take a look at what happens\nin bank 1 after four years, you're going to end up with $16. In bank 2 after four years, you're going to end\nup with more money. You're going to end\nup with $25.63. In bank 3, you're going to end up with quite a lot of money. You're going to end\nup with $31.57. Definitely bank 3 is much\nbetter than bank 1 and bank 2. Again, let's recall\nthat after one year, in bank 1 you get\n1 plus 1 to the 1, bank 2, you get 1 plus 1/2 to 2 and in bank 3 get 1\nplus 1/3 to the 3. This is super important. Now, could you imagine a\nbetter bank than these three? Well, of course, the more subdivisions\nyou have, the better. Let's look at bank 12. Bank 12 gives you 1/12 of\nyour money every month. You start with $1\nnow, in a month, you have 1 plus\n1/12, in two months, 1 plus 1/12 squared and\nso on until in 12 months, you have 1 plus 1/12 to\nthe 12th and that's $2.61. You did much better. In general, bank n, would have\nn subdivisions. It subdivides the year into\nn intervals and every time, it gives you one\nnth of your money. You start with one\non the first day. After first interval, 1 plus 1 over n. After the\nsecond interval, 1 plus 1 over n squared. If this is not super\nclear, let me elaborate. 1 plus 1 over n is the money you have and you add an nth of that. You have 1 plus 1\nover n plus 1 over n, 1 plus 1 over n and that is the money you have plus\nthe interests you accrue. That factors is 1\nplus 1 over n square. After three intervals, you\nhave 1 plus 1 over n cubed. After k intervals, you have 1 plus 1 over\nn to the k. Eventually, after the end of the year, you have 1 plus 1 over n to\nthe n. N can be really large, so you could potentially accrue money every second or\nevery millisecond. In summary, bank n\ndoes the following, it gives you 1 plus 1 over n to the n at\nthe end of the year because it returns one nth of your money and times a year. Now imagine that you go\naround looking for banks. There's bank 1, there's\nbank 2, there's 3. Bank 4 gives you a quarter of your money\nfour times a year. Bank 5 gives you a fifth\nof your money five times a year and so on until\nthe end of the universe. Here's bank 12 that gives you 1/12 of your money every month. Here's bank 365, that gives you 1/365 of your money every day. What's at the very\nend of the years? What is this bank? Well, this bank, let's call\nit bank infinity. What bank infinity does is it pretty much gives\nyou instant interests. One infinity of your money\ninfinity times a year. Let's see how much\nmoney bank infinity would give you at\nthe end of the year. Bank 1 ends up with\n1 plus 1 to the 1. Bank 2 with 1 plus 1/2 to the 2. Bank 3 is 1 plus 1/3 to the 3 and 12 is 1 plus\n1/12 to the 12. Bank 365 is 1 plus\n1/365 to the 365, which already is 2.7145. Bank infinity gives you, well, this is not a good\nmathematical expression, 1 plus 1 over infinity\nto the infinity. But it's basically the number that all these numbers\nconverge to 2.25, 2.37, 2.613, 2.7145, etc. Gets closer and closer\nto a particular number. That number is 2.7182812\nor the number e. So e is the amount of money that you get at the end of the\nyear in bank infinity because bank infinity gives you an infinitesimal if your\nmoney every single instant. That is one way to\nlook at the number e."

# Split the text based on space and newline
split_text = re.split(r"[\s\n]+", text)

n_flashcards = int(len(split_text) / 65)
n_flashcards = max(3, n_flashcards)


messages = [
    {
        "role": "system",
        "content": "You are a teacher who is writing anki cards for his students.",
    },
    {
        "role": "user",
        # "content": f"Please write an anki card on the following text:\n{text}",
        "content": prompt_1 % (text, 20),
    },
]


input_ids = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)

terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1] :]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Here is the output in CSV format:

Front;Back;Number
"What is Euler's number defined as?";"By the numerical value 2.71828182, etc, or using the expression 1 plus 1 over n to the n.";1
"What is the property of the function f(x) = e^x?";"It is its own derivative, i.e., the derivative of f(x) is also f(x).";2
"What is the best bank among Bank 1, Bank 2, and Bank 3?";"Bank 3, which gives you 1/3 of your money three times a year, is the best bank.";3
"What is the formula for the money you have in Bank 3 after one year?";"1 plus 1/3 to the 3.";4
"What is the formula for the money you have in Bank n after one year?";"1 plus 1 over n to the n.";5
"What is the formula for the money you have in Bank infinity after one year?";"1 plus 1 over infinity to the infinity, which converges to the number e.";6
"What is the number e?";"The number e is 2.71828182, etc, and it is the amount of money you get at the end of the year in bank infinity.";7
"What is the way to define the number e using interest rat

In [3]:
256 * 4

1024

In [4]:
import re

# Split the text based on space and newline
split_text = re.split(r"[\s\n]+", text)

print(len(split_text) / 60)

26.266666666666666


In [5]:
tokenizer.decode(response, skip_special_tokens=True)

'Here is the output in CSV format:\n\nFront;Back;Number\n"What is Euler\'s number defined as?";"By the numerical value 2.71828182, etc, or using the expression 1 plus 1 over n to the n.";1\n"What is the property of the function f(x) = e^x?";"It is its own derivative, i.e., the derivative of f(x) is also f(x).";2\n"What is the best bank among Bank 1, Bank 2, and Bank 3?";"Bank 3, which gives you 1/3 of your money three times a year, is the best bank.";3\n"What is the formula for the money you have in Bank 3 after one year?";"1 plus 1/3 to the 3.";4\n"What is the formula for the money you have in Bank n after one year?";"1 plus 1 over n to the n.";5\n"What is the formula for the money you have in Bank infinity after one year?";"1 plus 1 over infinity to the infinity, which converges to the number e.";6\n"What is the number e?";"The number e is 2.71828182, etc, and it is the amount of money you get at the end of the year in bank infinity.";7\n"What is the way to define the number e using 

In [6]:
from io import StringIO


def clean_response(response_text):
    """Clean Response Text
    Args:
        response_text (str): response text from model
    Returns:
        response_df (df): cleaned response df
    """

    # clear response text
    # delete everything before Front;Back;Number
    response_text = response_text[response_text.find("Front;Back;Number") :]

    # Use StringIO to create a file-like object from the string
    _data = StringIO(response_text)

    # Read the data into a pandas dataframe
    df = pd.read_csv(_data, delimiter=";", quotechar='"')

    return df


clean_response(response_text=tokenizer.decode(response, skip_special_tokens=True))

NameError: name 'StringIO' is not defined

In [ ]:
response_text.find("Front;Back;Number")

79

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a a priest.",
    },
    {
        "role": "user",
        # "content": f"Please write an anki card on the following text:\n{text}",
        # "content": prompt_1 % (text, 20),
        "content": "How does LLAMA-3 filter-out inappropriate questions",
    },
]


input_ids = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)

terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1] :]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


My child, I must confess that I am not familiar with this "LLAMA-3" you speak of. As a priest, my duties lie in the realm of spiritual guidance and pastoral care, not in the realm of artificial intelligence or technology.

However, I can tell you that as a priest, I am trained to discern and respond to questions that are respectful and relevant to the spiritual journey. I believe that a good priest should be able to navigate complex conversations and provide guidance that is informed by faith and compassion.

If I were to assume that LLAMA-3 is a tool designed to facilitate spiritual inquiry, I would imagine that it would employ various mechanisms to filter out inappropriate questions. Here are a few possibilities:

1. Natural Language Processing (NLP): LLAMA-3 might use NLP algorithms to analyze the language and tone of the question, identifying and flagging any that contain profanity, aggression, or other inappropriate content.
2. Contextual understanding: The tool might be designed 